In [1]:
import sys
import os

# Check current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

# Change to your project root directory
project_root = '/home/razvansavin/Proiecte/flexiai'
os.chdir(project_root)
print(f"Changed Directory to: {os.getcwd()}")

# Add project root directory to sys.path
sys.path.append(project_root)
print(f"Project root added to sys.path")

Current Directory: /home/razvansavin/Proiecte/flexiai/examples/Code examples
Changed Directory to: /home/razvansavin/Proiecte/flexiai
Project root added to sys.path


In [2]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


1. Setup and Initialization

In [3]:
import json
import logging
from flexiai.core.flexiai_client import FlexiAI
from flexiai.config.logging_config import setup_logging
from pydantic import BaseModel


# Initialize logging and FlexiAI client
setup_logging(
    root_level=logging.INFO, 
    file_level=logging.INFO, 
    console_level=logging.INFO, 
    enable_file_logging=True, 
    enable_console_logging=True
    )
flexiai = FlexiAI()
logger = flexiai.logger
completions_manager = flexiai.completions_manager


2024-08-10 01:18:19,235 - INFO - task_manager.py - Initializing TaskManager
2024-08-10 01:18:19,237 - INFO - function_mapping.py - User directory detected: /home/razvansavin/Proiecte/flexiai/user_flexiai_rag
2024-08-10 01:18:19,238 - INFO - run_manager.py - Initialized RunManager with personal functions: []
2024-08-10 01:18:19,239 - INFO - run_manager.py - Initialized RunManager with assistant functions: []
2024-08-10 01:18:19,240 - INFO - function_mapping.py - Attempting to import module: user_task_manager
2024-08-10 01:18:19,242 - INFO - function_mapping.py - Attempting to import module: user_helpers
2024-08-10 01:18:19,243 - INFO - function_mapping.py - Attempting to import module: user_function_mapping
2024-08-10 01:18:19,244 - INFO - user_function_mapping.py - Registering user tasks
2024-08-10 01:18:19,244 - INFO - function_mapping.py - Successfully registered user functions from user_function_mapping
2024-08-10 01:18:19,246 - INFO - task_manager.py - User-defined tasks loaded suc

Current working directory: /home/razvansavin/Proiecte/flexiai
Log directory '/home/razvansavin/Proiecte/flexiai/logs' created/exists.


2. Test Simple Chat Completion
> This will test the basic completion functionality without any structured output

In [4]:
# Define a simple prompt for testing
simple_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather like today?"}
]

# Perform the simple chat completion
simple_result = completions_manager.simple_chat_completion(
    model="gpt-4o-mini",
    messages=simple_messages
)

# Output the result
print("Simple Completion Result:", simple_result)


2024-08-10 01:18:19,259 - INFO - completions_manager.py - Performing simple chat completion with model gpt-4o-mini
2024-08-10 01:18:20,494 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-10 01:18:20,500 - INFO - completions_manager.py - Received message content: I'm sorry, but I can't provide real-time weather updates. However, you can check a reliable weather website or app for the latest information on today's weather. If you tell me your location, I can guide you on where to find the weather details!


Simple Completion Result: I'm sorry, but I can't provide real-time weather updates. However, you can check a reliable weather website or app for the latest information on today's weather. If you tell me your location, I can guide you on where to find the weather details!


3. Test Structured Chat Completion
> This will test the structured output functionality using a predefined schema.

In [5]:
# {
#   "type": "object",
#   "required": ["steps", "final_answer"],
#   "properties": {
#     "steps": {
#       "type": "array",
#       "items": {
#         "type": "object",
#         "required": ["explanation", "output"],
#         "properties": {
#           "output": {"type": "string"},
#           "explanation": {"type": "string"}
#         },
#         "additionalProperties": false
#       }
#     },
#     "final_answer": {"type": "string"}
#   },
#   "additionalProperties": false
# }


In [6]:
# Define the schema for the structured output
math_response_schema = {
    "type": "object",
    "required": ["steps", "final_answer"],
    "properties": {
        "steps": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["explanation", "output"],
                "properties": {
                    "output": {"type": "string"},
                    "explanation": {"type": "string"}
                },
                "additionalProperties": False
            }
        },
        "final_answer": {"type": "string"}
    },
    "additionalProperties": False
}

# Define the messages
structured_messages = [
    {"role": "system", "content": "You are a math assistant."},
    {"role": "user", "content": "Explain how to calculate the average of the numbers 4, 8, 6, 5, and 3."}
]

# Perform the structured chat completion
structured_result = completions_manager.structured_chat_completion(
    model="gpt-4o-mini",
    messages=structured_messages,
    schema_name="math_response",
    schema=math_response_schema
)

# Parse the JSON string into a Python dictionary
structured_output = json.loads(structured_result)
print(*'=')
# Print the output in a readable format
print("Structured Completion Result:")
print(f"Final Answer: {structured_output['final_answer']}\n")

print("Steps:")
for step in structured_output['steps']:
    print(f"Explanation: {step['explanation']}")
    print(f"Output: {step['output']}\n")

2024-08-10 01:18:20,521 - INFO - completions_manager.py - Performing structured chat completion with model gpt-4o-mini
2024-08-10 01:18:24,448 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-10 01:18:24,454 - INFO - completions_manager.py - Received structured output: {"steps":[{"output":"Add all the numbers together: 4 + 8 + 6 + 5 + 3","explanation":"To find the average, you first need to find the sum of all the numbers."},{"output":"4 + 8 + 6 + 5 + 3 = 26","explanation":"When you perform the addition, the total sum of the numbers is 26."},{"output":"Count how many numbers there are: 5","explanation":"Next, you count how many numbers you are averaging. In this case, there are 5 numbers."},{"output":"Divide the sum by the count: 26 / 5","explanation":"To find the average, you divide the total sum by the number of values."},{"output":"26 / 5 = 5.2","explanation":"When you divide 26 by 5, you get 5.2."}],"final_answer":"The av

=
Structured Completion Result:
Final Answer: The average of the numbers 4, 8, 6, 5, and 3 is 5.2.

Steps:
Explanation: To find the average, you first need to find the sum of all the numbers.
Output: Add all the numbers together: 4 + 8 + 6 + 5 + 3

Explanation: When you perform the addition, the total sum of the numbers is 26.
Output: 4 + 8 + 6 + 5 + 3 = 26

Explanation: Next, you count how many numbers you are averaging. In this case, there are 5 numbers.
Output: Count how many numbers there are: 5

Explanation: To find the average, you divide the total sum by the number of values.
Output: Divide the sum by the count: 26 / 5

Explanation: When you divide 26 by 5, you get 5.2.
Output: 26 / 5 = 5.2



4. Test Function Calling Completion

> This will test the function calling capability of the model.

In [7]:
import logging
from flexiai.core.flexiai_client import FlexiAI
from flexiai.config.logging_config import setup_logging


def initialize_flexiai():
    """Initializes FlexiAI client and logger."""
    setup_logging()
    flexiai = FlexiAI()
    logger = flexiai.logger
    return flexiai, logger


def check_function_call(function_call):
    """Simulates the execution of a function if a function call is detected."""
    if function_call:
        print("Function Call Detected:")
        print(function_call)

        # Simulate function execution result
        simulated_function_result = {
            "location": "New York",
            "temperature": "15°C",
            "condition": "Partly cloudy"
        }

        return str(simulated_function_result)
    else:
        print("No function call detected.")
        return None


def handle_follow_up(completions_manager, model, messages, function_call_result):
    """Handles the follow-up interaction after simulating the function execution."""
    if function_call_result:
        follow_up_messages = messages + [{"role": "assistant", "content": function_call_result}]
        follow_up_result = completions_manager.simple_chat_completion(model=model, messages=follow_up_messages)
        print("Follow-Up Result:", follow_up_result)
    else:
        print("Initial Result:", function_call_result)


def main():
    # Initialize FlexiAI and get the logger and manager
    flexiai, logger = initialize_flexiai()
    completions_manager = flexiai.completions_manager

    # Define the function that the model can call
    functions = [
        {
            "name": "get_weather",
            "description": "Fetches the weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["C", "F"]}
                },
                "required": ["location", "unit"]
            }
        }
    ]

    # Define the messages
    function_calling_messages = [
        {"role": "system", "content": "You are an assistant that can fetch weather information."},
        {"role": "user", "content": "Get the weather in New York in Celsius."}
    ]

    # Perform the function calling chat completion
    function_call_result, function_call = completions_manager.function_calling_completion(
        model="gpt-4o-mini",
        messages=function_calling_messages,
        functions=functions
    )

    # Check and handle the function call
    simulated_result = check_function_call(function_call)

    # Handle follow-up interaction
    handle_follow_up(completions_manager, "gpt-4o-mini", function_calling_messages, simulated_result)


if __name__ == "__main__":
    main()


2024-08-10 01:18:24,484 - INFO - task_manager.py - Initializing TaskManager
2024-08-10 01:18:24,527 - INFO - function_mapping.py - User directory detected: /home/razvansavin/Proiecte/flexiai/user_flexiai_rag
2024-08-10 01:18:24,527 - INFO - run_manager.py - Initialized RunManager with personal functions: []
2024-08-10 01:18:24,529 - INFO - run_manager.py - Initialized RunManager with assistant functions: []
2024-08-10 01:18:24,530 - INFO - function_mapping.py - Attempting to import module: user_task_manager
2024-08-10 01:18:24,532 - INFO - function_mapping.py - Attempting to import module: user_helpers
2024-08-10 01:18:24,533 - INFO - function_mapping.py - Attempting to import module: user_function_mapping
2024-08-10 01:18:24,534 - INFO - user_function_mapping.py - Registering user tasks
2024-08-10 01:18:24,535 - INFO - function_mapping.py - Successfully registered user functions from user_function_mapping
2024-08-10 01:18:24,536 - INFO - task_manager.py - User-defined tasks loaded suc

Current working directory: /home/razvansavin/Proiecte/flexiai
Log directory '/home/razvansavin/Proiecte/flexiai/logs' created/exists.


2024-08-10 01:18:26,573 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-10 01:18:26,576 - INFO - completions_manager.py - Function call made: FunctionCall(arguments='{"location":"New York","unit":"C"}', name='get_weather')
2024-08-10 01:18:26,576 - INFO - completions_manager.py - Received message content: None
2024-08-10 01:18:26,577 - INFO - completions_manager.py - Performing simple chat completion with model gpt-4o-mini


Function Call Detected:
FunctionCall(arguments='{"location":"New York","unit":"C"}', name='get_weather')


2024-08-10 01:18:27,624 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-10 01:18:27,625 - INFO - completions_manager.py - Received message content: I'm unable to provide real-time weather information. However, you can easily check the current weather in New York by using a weather website or app. Typically, New York experiences a range of temperatures depending on the season, so be sure to check for up-to-date information!


Follow-Up Result: I'm unable to provide real-time weather information. However, you can easily check the current weather in New York by using a weather website or app. Typically, New York experiences a range of temperatures depending on the season, so be sure to check for up-to-date information!


5. Test Refusal Handling

> This test checks how the model handles scenarios where it refuses to fulfill a request.

In [10]:
# Define a message that might cause a refusal
refusal_messages = [
    {"role": "system", "content": "You are a strict assistant."},
    {"role": "user", "content": "Tell me how to hack a website."}
]

# Perform the structured chat completion
refusal_result = completions_manager.simple_chat_completion(
    model="gpt-4o-mini",
    messages=refusal_messages
)

# Handle and print the refusal
refusal_reason = completions_manager.handle_refusals(refusal_result)
print("Refusal Reason:", refusal_reason)


2024-08-10 01:18:55,755 - INFO - completions_manager.py - Performing simple chat completion with model gpt-4o-mini
2024-08-10 01:18:56,364 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-10 01:18:56,366 - INFO - completions_manager.py - Received message content: I’m sorry, I can’t assist with that.
2024-08-10 01:18:56,366 - WARNING - completions_manager.py - Received string response: I’m sorry, I can’t assist with that.


Refusal Reason: Potential refusal or error in response


In [9]:
import json
from openai import OpenAI
import tiktoken

def make_parameters(frogs: list[str]):
    assert len(frogs) <= 25, "OpenAI can only handle 25 frogs at once with this schema"
    messages = [
        {"role": "system", "content": "You are a biologist specializing in aquatics who has been tasked with describing frogs."},
        {"role": "user", "content": "For each frog, please describe its color, temperament, and taste in two words or less. Reply in JSON."},
    ]
    json_schema = {
        "type": "object",
        "properties": {
            v: {
                "type": "object",
                "properties": {
                    "color": {"type": "string"},
                    "temperament": {"type": "string"},
                    "taste": {"type": "string"},
                },
                "required": ["color","temperament","taste"],
                "additionalProperties": False,
            }
            for v in frogs
        },
        "required": frogs,
        "additionalProperties": False,
    }
    
    return {
        "messages": messages,
        "model": "gpt-4o-mini",
        "temperature": 0.1,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "summarization",
                "strict": True,
                "schema": json_schema,
            }
        }
    }

client = OpenAI()
parameters = make_parameters(["green tree frog", "poison arrow frog", "peeper", "bullfrog"])

encoder = tiktoken.encoding_for_model(parameters["model"])
answer = client.chat.completions.create(**parameters)

message = answer.choices[0].message.content
local_output_tokens = len(encoder.encode(message))


# Print the result
print("Result from API:")
print(json.dumps(json.loads(message), indent=2))  # Pretty print the JSON response


2024-08-10 01:18:30,170 - INFO - _client.py - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Result from API:
{
  "green tree frog": {
    "color": "Bright green",
    "temperament": "Calm",
    "taste": "Mildly sweet"
  },
  "poison arrow frog": {
    "color": "Vibrant blue",
    "temperament": "Aggressive",
    "taste": "Toxic"
  },
  "peeper": {
    "color": "Brownish green",
    "temperament": "Vocal",
    "taste": "Unremarkable"
  },
  "bullfrog": {
    "color": "Olive green",
    "temperament": "Loud",
    "taste": "Savory"
  }
}
